In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(palette="deep", font_scale=1.1, rc={"figure.figsize": [8, 5]})
import random
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split


In [4]:
# read the data
df = pd.read_csv('./data/logged_transformed_data.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.shape

(5777, 17)

In [9]:
df.head()

,TONDUR,EngSpDist_K1,EngSpDist_K2,EngSpDist_K3,EngSpDist_K4,EngSpDist_K5,EngSpDist_K6,EngSpDist_K7,AmbTempDist_K5,AmbTempDist_K6,AmbTempDist_K7,AmbTempDist_K8,AmbTempDist_K9,AmbTempDist_K10,MILEAGE,LABEL
0,14.038220,12.386195,13.002472,12.346563,9.849897,8.147491,7.397562,6.619273,2.397895,5.762051,6.033086,5.049856,0.000000,0.000000,10.463360,0
1,14.613686,13.674723,13.667726,11.945884,9.450003,7.967869,6.859195,6.148255,2.708050,6.171701,6.173786,5.123964,0.693147,0.000000,9.613135,0
2,15.049144,13.620402,14.485669,12.812301,10.168882,7.281799,5.477300,4.527209,1.945910,6.517671,6.901737,5.568345,1.098612,0.000000,10.537044,0
3,13.329598,11.841770,12.250756,11.573650,9.223740,7.014724,5.445443,4.412798,0.000000,5.135798,5.690359,4.727388,1.098612,0.000000,8.861492,0
4,14.576953,13.589420,13.500192,12.399762,9.494737,7.772963,6.705272,5.835103,2.564949,5.883322,6.621406,5.438079,1.098612,0.693147,9.887409,0


#### For the modeling, baseline needs to established in order to evaluate the performance of the model. For simplicity, I will only consider models from three types: **Linear**, **Non-Linear** and **Ensemble** methods.

### Baseline without upsampling, feature selection (RF, Perm method), and hyperparameters tuning 


In [ ]:
from sklearn.linear_model import LogisticRegression
